# 04 - NER with spaCy (pre-trained + small custom training)


In [ ]:
# !pip install -q spacy
# !python -m spacy download en_core_web_sm

import spacy
from spacy import displacy
from pathlib import Path
import os
os.makedirs("../models", exist_ok=True)
os.makedirs("../figures", exist_ok=True)


In [ ]:
nlp = spacy.load("en_core_web_sm")
text = "Apple is looking at buying U.K. startup for $1 billion. Elon Musk visited Nigeria in 2024."
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.label_)
displacy.render(doc, style="ent", jupyter=True)


In [ ]:
import random
TRAIN_DATA = [
    ("She was prescribed Amoxicillin for her infection.", {"entities": [(20, 31, "MEDICATION")]}),
    ("Take two tablets of Paracetamol daily.", {"entities": [(19, 30, "MEDICATION")]}),
    ("Ibuprofen should not be mixed with alcohol.", {"entities": [(0, 9, "MEDICATION")]}),
]

nlp = spacy.load("en_core_web_sm")
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe("ner")
ner.add_label("MEDICATION")

optimizer = nlp.resume_training()
for i in range(20):
    random.shuffle(TRAIN_DATA)
    losses = {}
    for text, annotations in TRAIN_DATA:
        nlp.update([ (text, annotations) ], sgd=optimizer, losses=losses)
    if i % 5 == 0:
        print("Iteration", i, "losses", losses)


In [ ]:
doc = nlp("He took Ibuprofen after dinner.")
for ent in doc.ents:
    print(ent.text, ent.label_)
# save model
output_dir = Path("../models/custom_ner_model")
nlp.to_disk(output_dir)
print("Saved custom NER model to", output_dir)


Notes:
- Include spaCy visualization screenshot in report.
- Saved custom model at /models/custom_ner_model
